In [15]:
import utils
import importlib

importlib.reload(utils)

<module 'utils' from '/data/tongyx361/reward-by-prm800k/src/utils.py'>

In [2]:
from transformers import LlamaForCausalLM, GenerationConfig
import random
import torch

In [11]:
utils.init_logging()
logger = utils.get_logger(__name__)
logger.info("test")

2023-09-13 07:49:59.035 [INFO] test
2023-09-13 07:49:59.035 [INFO] test
2023-09-13 07:49:59.035 [INFO] test
2023-09-13 07:49:59.035 [INFO] test
2023-09-13 07:49:59.035 [INFO] test
2023-09-13 07:49:59.035 [INFO] test


<Logger __main__ (INFO)>


In [18]:
# utils.set_gpu_ids([6])
gpu_id = "6"
device = f"cuda:{gpu_id}"

os.environ["CUDA_VISIBLE_DEVICES"] = 6


In [16]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
# tokenizer = LlamaTokenizer.from_pretrained(tokenizer_name_or_path)
tokenizer = utils.get_complete_tokenizer()

head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32000
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}
num_added_tokens = 1
head_tokens = <unk><s></s> 
tokenizer.vocab_size = 32000
len(tokenizer.vocab) = 32001
tokenizer.special_tokens_map = {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}
LlamaTokenizerFast(name_or_path='/data/tongyx361/.cache/huggingface/hub/models--hf-internal-testing--llama-tokenizer/snapshots/99eceeba6e8289bee767f0771166b5917e70e470', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)
{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}


In [19]:
model = LlamaForCausalLM.from_pretrained(
    utils.default_7b_model_path, low_cpu_mem_usage=True
).to(device)

embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

print(model)
print(model.config)
print(model.generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 6; 23.70 GiB total capacity; 23.06 GiB already allocated; 3.69 MiB free; 23.06 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def sparse_print(*arg, **kwargs):
    print(*arg, **kwargs, sep="\n", end="\n\n")


def generate(prompt_or_list, generation_config):
    if not isinstance(generation_config, GenerationConfig):
        if isinstance(generation_config, dict):
            generation_config = GenerationConfig.from_dict(generation_config)

    inputs = tokenizer(prompt_or_list, padding=True, return_tensors="pt").to(device)
    sparse_print("inputs:")
    sparse_print(inputs)

    outputs = model.generate(**inputs, generation_config=generation_config)
    sparse_print(outputs)

    if not generation_config.return_dict_in_generate:
        decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        sparse_print(decoded_outputs)

        return outputs
    else:  # generation_config.return_dict_in_generate == True
        outputs_len = outputs["sequences"].shape[0]

        example_idx = random.randint(0, outputs_len - 1)
        sparse_print(f"example_idx = {example_idx}")

        if outputs_len > 5:
            example_decoded_output = tokenizer.decode(
                outputs["sequences"][example_idx], skip_special_tokens=True
            )
            sparse_print(example_decoded_output)
        else:  # outputs_len <= 5
            decoded_outputs = tokenizer.batch_decode(
                outputs["sequences"], skip_special_tokens=True
            )
            sparse_print(decoded_outputs)

        if generation_config.output_scores:
            example_scores = outputs["scores"][example_idx]
            sparse_print(example_scores.shape)
            sparse_print(example_scores)

            example_probs = torch.nn.functional.softmax(example_scores, dim=-1)
            sparse_print(example_probs[example_idx])
            sparse_print(sum(example_probs[example_idx]))

            # slow cpu
            # for idx, prob in enumerate(first_probs[0]):
            #     if prob > 1e-5:
            #         print(f"token_id: {idx}, token: {tokenizer.decode([idx])}, prob: {prob}")

            # fast gpu
            # non_zero_idxs = torch.gt(first_probs.view(-1),1e-1)
            threshold = 0.1
            mask = example_probs > threshold
            non_zero_idxs = torch.nonzero(mask)
            sparse_print(non_zero_idxs)
            non_zero_vals = example_probs.view(-1)[non_zero_idxs]
            sparse_print(non_zero_vals)

        return outputs

In [ ]:
# test
generation_config = {
    "do_sample": True,
    # "temperature": 0.8,
    # "top_p": 0.95,
    "temperature": 0.7,
    "top_p": 0.9,
    "max_new_tokens": 10,
    "return_dict_in_generate": True,
}

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

outputs = generate(prompt_or_list=prompts, generation_config=generation_config)

decoded_outputs = tokenizer.batch_decode(**outputs, skip_special_tokens=True)

for prompt, decoded_output in zip(prompts, decoded_outputs):
    print(f"Prompt    : {prompt}")
    print(f"Generation: {decoded_output[len(prompt):]}")
    print()

inputs:

{'input_ids': tensor([[    1, 15043, 29892,   590,  1024,   338, 32000, 32000],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338],
        [    1,   450,  7483,   310,  3444,   338, 32000, 32000],
        [    1,   450,  5434,   310,   319, 29902,   338, 32000]],
       device='cuda:3'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0]], device='cuda:3')}

SampleDecoderOnlyOutput(sequences=tensor([[    1, 15043, 29892,   590,  1024,   338, 32000, 32000, 30488, 30488,
         30488, 30488, 30488, 30488, 30488, 30488, 30488, 30488],
        [    1,   450,  6673,   310,   278,  3303,  3900,   338,   278,  2343,
           310,  2106,   322,  2343,   310,  5874,   310,   278],
        [    1,   450,  7483,   310,  3444,   338, 32000, 32000, 30488,  3681,
         29889,   450,  4272,   338,  5982,   297,   278,  6641],
        [    1,   450,  5434,   310,   319, 

In [ ]:
# debug
print(generation_config.return_dict_in_generate)

False


In [ ]:
problem = "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?"
steps = [
    "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
    "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
    "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
    "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
]

rating2word = {1: "positive", -1: "negative", 0: "neutral"}
rating_words = list(rating2word.values())
rating_token_ids = tokenizer(rating_words, add_special_tokens=False).input_ids
print(rating_token_ids)

# constraint = transformers.Constraint(type="must_include", token="apple")

# config
generation_config = {
    "do_sample": False,
    "max_new_tokens": 1,
    "force_words_ids": rating_token_ids,
    "num_beams": 2,
    "remove_invalid_values": True,
    "return_dict_in_generate": True,
    "output_scores": True,
}

In [ ]:
generate(problem + steps[0], generation_config=generation_config)